This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Libraries
https://huggingface.co/docs/transformers/training

## Install

In [1]:
!pip install datasets

## Import

In [4]:
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns', None)

In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

global_seed = 2022
seed_torch(global_seed)

## Get device

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

# Load tokenized data

From my [other notebook](https://www.kaggle.com/code/khairulislam/tokenize-jigsaw-comments). The dataset is tokenized from the [Jigsaw competition]( https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification) and [all_data.csv](https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data?select=all_data.csv)

In [8]:
text = 'comment_text'
target = 'labels'
root = '/kaggle/input/tokenize-jigsaw-comments/'

In [9]:
import pickle
    
with open(root + 'test.pkl', 'rb') as input_file:
    test_all_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'train_undersampled.pkl', 'rb') as input_file:
    train_all_tokenized = pickle.load(input_file)
    input_file.close()

In [10]:
train_tokenized = train_all_tokenized.remove_columns(['id'])
test_tokenized = test_all_tokenized.remove_columns(['id'])

# Model

BERT (Bidirectional Encoder Representations from Transformers) is a state of the art approach to various NLP tasks. It uses a Transformer architecture and relies heavily on the concept of pre-training.

We'll use a pre-trained BERT-base model, provided in huggingface [transformers](https://github.com/huggingface/transformers) repo. It gives us a pytorch implementation for the classic BERT architecture, as well as a tokenizer and weights pre-trained on a public English corpus (Wikipedia).

Please follow these [installation instrucitons](https://github.com/huggingface/transformers#installation) before proceeding.

In [11]:
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
from transformers import AutoModelForSequenceClassification

def load_pretrained_model(model_name, num_labels):
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
    total_params = 0
    trainable_params = 0

    for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

    for layer in trainable_layers:
        for p in layer.parameters():
            p.requires_grad = True
            trainable_params += p.numel()

    print(f"Total parameters count: {total_params}") # ~108M
    print(f"Trainable parameters count: {trainable_params}") # ~7M

    return model

In [12]:
num_labels = 2
# model_name = "bert-base-uncased"
model_name = 'prajjwal1/bert-small'

# Training

## Utils

In [14]:
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

sigmoid = torch.nn.Sigmoid()

# https://huggingface.co/docs/datasets/metrics
def calculate_result(labels, probs, threshold=0.5):
    preds = np.where(probs >= threshold, 1, 0)
    return {
        'accuracy': np.round(accuracy_score(labels, preds), 4),
        'f1': np.round(f1_score(labels, preds), 4),
        'auc': np.round(roc_auc_score(labels, probs), 4)
    }

def evaluate(model, test_dataloader, epoch):    
    model.eval()

    losses, total_labels = [], []
    total_probs = torch.tensor([], dtype=torch.float32)
    progress_bar = tqdm(range(len(test_dataloader)), desc=f'Epoch {epoch} (Test)')
    
    for batch in test_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        
        probs = sigmoid(outputs.logits.detach().cpu())[:, 1]
        labels = inputs[target].detach().cpu().numpy()
        
        losses.append(loss.item())
        total_probs = torch.cat((total_probs, probs), dim=0)
        total_labels.extend(labels)
        
        progress_bar.update(1)
        progress_bar.set_postfix(
            loss=np.round(np.mean(losses), 4), 
            f1=np.round(f1_score(total_labels, total_probs>=0.5), 4)
        )
    
    model.train()
    test_result = calculate_result(total_labels, total_probs)
    return np.mean(losses), test_result, total_probs

def train(model, train_dataloader, epoch):
    losses, total_labels = [], []
    total_probs = torch.tensor([], dtype=torch.float32)
    progress_bar = tqdm(range(len(train_dataloader)), desc=f'Epoch {epoch} (Train)')

    for step, data in enumerate(train_dataloader):
        optimizer.zero_grad()

        inputs = {k: v.to(device) for k, v in data.items()}
        outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

        # targets = data[target].to(device, dtype = torch.long)
        # loss = loss_function(outputs.logits, targets)
        loss = outputs[0]

        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        # preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
        probs = sigmoid(outputs.logits.detach().cpu())[:, 1]
        labels = inputs[target].detach().cpu().numpy()
        
        total_probs = torch.cat((total_probs, probs), dim=0)
        total_labels.extend(labels)

        progress_bar.update(1)
        progress_bar.set_postfix(
            loss=np.round(np.mean(losses), 4), 
            f1=np.round(f1_score(total_labels, total_probs>=0.5), 4)
        )


    train_loss = np.mean(losses)
    train_result = calculate_result(np.array(total_labels), np.array(total_probs))

    return train_loss, train_result, total_probs

def dump_results(epoch=None):
    train_df = pd.DataFrame({'id':train_all_tokenized['id'], 'labels':train_all_tokenized[target], 
      'probs': train_probs, 'split':['train']* len(train_all_tokenized)
    })
    test_df = pd.DataFrame({'id':test_all_tokenized['id'], 'labels':test_all_tokenized[target], 
      'probs': test_probs, 'split':['test']* len(test_all_tokenized)
    })

    total_df = pd.concat([train_df, test_df],ignore_index=True)

    if epoch is None:
        total_df.to_csv('results.csv', index=False)
    else:
        total_df.to_csv(f'results_{epoch}.csv', index=False)
        
    # return preds_df

## Data loader

In [ ]:
BATCH_SIZE = 128

train_dataloader = DataLoader(train_tokenized, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_tokenized, batch_size=BATCH_SIZE)

## Model and optimizer

In [25]:
model = load_pretrained_model(model_name, num_labels)

# Define optimizer
LEARNING_RATE = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
EPOCHS = 3

# https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

## Loop

In [27]:
# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train().to(device)

for epoch in range(1, EPOCHS+1):
    gc.collect()
    
    train_loss, train_result, train_probs = train(model, train_dataloader, epoch)
    test_loss, test_result, test_probs = evaluate(model, test_dataloader, epoch)

    print(
      f"Epoch: {epoch} | "
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Test loss: {test_loss:.3f} | "
      f"Test result: {test_result} | "
    )
    dump_results(epoch)
    lr_scheduler.step()

|Model|Train Time|Learning rate|batch size|epoch |          |  train |         |       | test    |       |
|-----|----------|-------------|----------|------|----------|--------|---------|-------|---------|-------|
|     |          |             |          |      |  loss    |  f1    |  auc    | loss  | f1      | auc   |
|bert small|10 min|     1e-5   |    128   |   1  |  0.446   | 0.7927 |  0.8698 | 0.376 |  0.382  | 0.9170|
|     | 5min     |      1e-5   |    128   |   2  |  0.371   | 0.8351 |  0.9126 | 0.362 |  0.395  | 0.9287|
|     | | | | | | | | | | |
|     |  12min   |      1e-3   |    128   |   1  |    0.377 | 0.8163 |  0.9082 | 0.452 |  0.4912 | 0.9337|
|     |  11min   |      1e-3   |    128   |   2  |    0.354 | 0.8215 |  0.9198 | 0.363 |  0.4791 | 0.9371|
|     |  11min   |      1e-3   |    128   |   3  |    0.348 | 0.8383 |  0.9229 | 0.419 |  0.4679 | 0.9388|
|     |  11min   |      1e-3   |    128   |   4  |    0.342 | 0.8428 |  0.9264 | 0.344 |  0.5287 | 0.9402|
|     | | | | | | | | | | |
|bert small with lr scheduler|11min|1e-3|128| 1  |    0.377 | 0.8261 |  0.9097 | 0.419 |  0.4013 | 0.9341|
|     |          |    1e-3     |    128   |   2  |    0.347 | 0.8472 |  0.9250 | 0.391 |  0.4877 | 0.9394|
|     |          |    1e-3     |    128   |   3  |    0.333 | 0.8530 |  0.9312 | 0.336 |  0.5167 | 0.9422|
|     |          |    1e-3     |    128   |   4  |    0.320 | 0.8573 |  0.9363 | 0.305 |  0.5329 | 0.9448|
|     | | | | | | | | | | |
|     |          |     1e-4    |    128   |   1  |  0.358   | 0.8403 |  0.9174 | 0.335 |  0.4418 | 0.9434|
|     |          |      1e-4   |    128   |   2  |  0.312   | 0.8654 |  0.9381 | 0.305 |  0.4645 | 0.9486|
|     |          |      1e-4   |    128   |   3  |  0.297   | 0.8720 |  0.9438 | 0.300 |  0.4666 | 0.9503|
|     |          |      1e-4   |    128   |   4  |  0.286   | 0.8771 |  0.9477 | 0.304 |  0.4628 | 0.9506|
